# Lab 1 - FrozenLake MDP: Part 1
# Assignment

- In this assignment you will learn how to tackle problems with limited state spaces.
- In particular we consider the FrozenLake MDP problem.

# Outline

- Part 0 introduces us to [gym](https://gym.openai.com/), an environment that allows us to test our reinforcement learning algorithms in various problems
- In Part 1, you will implement a policy iteration algorithm (HW1)
- In Part 2, you will implement Q-Learning and SARSA (in next homework HW2) 

# Deliverable

Regarding the Lab:

- Make sure your code runs from top to bottom without any errors.
- Your submitted Notebook must contain saved outputs.

In [1]:
import os
# You will need numpy and gym. You can try running the following lines to install them
# The assignment is tested on Python3.8 so in case you are having installation issues you might 
# want to try installing that version. 

# !{os.sys.executable} -m pip install numpy
# !{os.sys.executable} -m pip install gym
import gym
import numpy as np

# Part 0 - Introduction to Gym
- We look at [FrozenLake-v0 environment](https://gym.openai.com/envs/FrozenLake-v0/) in gym. 
- You don't need to write any code for this part
- you should still understand the code to help you solve Part 1 and Part 2

In [2]:
# Import the environment we will use in this assignment
# env=gym.make('FrozenLake-v0') 
env=gym.make('FrozenLake-v1',is_slippery=True)
# Show the model
print(f"Number of States {env.nS}, Number of Actions {env.nA}")
print(f"Reward range {env.reward_range}")

Number of States 16, Number of Actions 4
Reward range (0, 1)


In [3]:
env.reset() # reset the environment 

0

In [4]:
# visualize the current state
env.render()


SFFF
FHFH
FFFH
HFFG


In [5]:
# run a policy that chooses actions randomly 
env.reset()
n = 25
for i in range(n):
    a = env.action_space.sample() # Sample Random Action
    state, reward, finished, _ = env.step(a)
    if finished: break
        
print(f'Render State after {n} slots')
env.render()
print(f'Reached terminal state? {finished}')

Render State after 25 slots
  (Down)
SFFF
FHFH
FFFH
HFFG
Reached terminal state? True


In [6]:
env.reset() # Let's reset the state again
env.render()


SFFF
FHFH
FFFH
HFFG


# Part 1 - MDP and Planning: Implement Policy Iteration 
- In this part we will focus on methods that assume knowledge of the enivonment dynamics, in partucular you will implement Policy Iteration. 
- The environment model can be obtained through `env.P`

In [7]:
# No need to change anything here. Try to understand what happens 

# let's look at a random state-action pair and observe its transition characteristics
# you can re-run this cell to get a different state-action pair
random_state  = env.observation_space.sample()
random_action = env.action_space.sample()
# returns a list of tuples (probability,newstate,reward,is_terminal_state)
env.P[random_state][random_action] 

[(1.0, 11, 0, True)]

In [8]:
############################
# YOUR CODE HERE #
# Print all the terminal states in the environment.
# you can use env.P

terminal_states = set()
for state in range(env.nS):
    for action in range(env.nA):
        for possible_state in env.P[state][action]:
            if possible_state[3]:
                terminal_states.add(possible_state[1])

print(terminal_states)
        
############################

{5, 7, 11, 12, 15}


In [9]:
# Verify your solution (look at the positions where final states are)
env.render()


SFFF
FHFH
FFFH
HFFG



### Step A: Implement Policy Evaluation


In [10]:
def policy_evaluation(P, nS, nA, policy, gamma=0.9, tol=1e-3):
    """Evaluate the value function from a given policy.

    Parameters
    ----------
    P, nS, nA, gamma:
        defined at beginning of file
    policy: np.array[nS]
        The policy to evaluate. Maps states to actions.
    tol: float
        Terminate policy evaluation when
            max |value_function(s) - prev_value_function(s)| < tol
    Returns
    -------
    value_function: np.ndarray[nS]
        The value function of the given policy, where value_function[s] is
        the value of state s
    """

    value_function = np.zeros(nS)

    ############################
    # YOUR IMPLEMENTATION HERE #
    
    while(True):
        
        diff = 0
        for s in range(nS):
                        
            v = value_function[s] 
            value_function[s] = sum([p*(r+gamma*value_function[n]) for p,n,r,_ in env.P[s][policy[s]]])
            diff = max(diff,abs(v-value_function[s]))

        if diff < tol:
            break

    ############################
    return value_function

#### Evaluate random policies

In [11]:
# Test your policy_evaluation on 5 randomly generated deterministic policies
# print the value function of the policies

############################
# YOUR CODE HERE #


for i in range(5):
    random_policy = np.random.randint(env.nA,size=env.nS)
    print(f'-------- Policy {i}','-'*30)
    print(policy_evaluation(env.P, env.nS, env.nA, random_policy))

############################


-------- Policy 0 ------------------------------
[0.02592173 0.02506689 0.03320955 0.         0.03734309 0.
 0.07764633 0.         0.08814752 0.16909485 0.22570486 0.
 0.         0.24994353 0.5832928  0.        ]
-------- Policy 1 ------------------------------
[0.00443043 0.         0.         0.         0.01137315 0.
 0.         0.         0.03405614 0.08036977 0.16390021 0.
 0.         0.23405313 0.5464036  0.        ]
-------- Policy 2 ------------------------------
[0.00419281 0.0016827  0.02861212 0.01210817 0.00928153 0.
 0.05569844 0.         0.02710647 0.06383039 0.18583226 0.
 0.         0.26526162 0.55576615 0.        ]
-------- Policy 3 ------------------------------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
-------- Policy 4 ------------------------------
[0.00637233 0.00258793 0.00505027 0.00324463 0.01382438 0.
 0.00965438 0.         0.04034093 0.08116614 0.02724616 0.
 0.         0.243703   0.4878335  0.        ]


### Step B: Implement Policy Improvement

In [13]:
def policy_improvement(P, nS, nA, value_from_policy, policy, gamma=0.9):
    """
    Given the value function from policy improve the policy.

    Parameters
    ----------
    P, nS, nA, gamma:
        defined at beginning of file
    value_from_policy: np.ndarray
        The value calculated from the policy
    policy: np.array
        The previous policy.

    Returns
    -------
    new_policy: np.ndarray[nS]
        An array of integers. Each integer is the optimal action to take
        in that state according to the environment dynamics and the
        given value function.
    """
    new_policy = np.zeros(nS, dtype='int')
    ############################
    # YOUR IMPLEMENTATION HERE #
    
    for s in range(nS):
        
#         if s in terminal_states:
#                 continue

        new_policy[s] = np.argmax([sum([p*(r+gamma*value_from_policy[n]) for p,n,r,_ in env.P[s][a]])
                                 for a in range(nA)])
        
#         old_action = policy[s]
#         new_action = policy[s]
#         max_value = float('-inf')
#         for a in range(nA):
#             new_value = sum([p*(r+gamma*value_function[n]) for p,n,r,_ in env.P[s][a]])
#             if new_value > max_value:
#                 max_value = new_value
#                 new_action = a
#         new_policy[s] = new_action
            
    ############################
    return new_policy

In [14]:
# Print the value before and after policy improvements for 5 randomly generated policies

############################
# YOUR CODE HERE #

for i in range(5):
    random_policy = np.random.randint(env.nA,size=env.nS)
    print(f'-------- Policy {i}','-'*30)
#     print(random_policy)
    value_function = policy_evaluation(env.P, env.nS, env.nA, random_policy)
#     print(f'-------- Value {i}','-'*30)
    print(value_function)
    
    print(f'-------> Policy {i} IMPROVED','-'*21)
    new_policy = policy_improvement(env.P, env.nS, env.nA, value_function, random_policy)
#     print(new_policy)
#     print(f'-------- Value {i} IMPROVED','-'*21)
    print(policy_evaluation(env.P, env.nS, env.nA, new_policy))

############################



-------- Policy 0 ------------------------------
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.14251    0.
 0.         0.         0.47584333 0.        ]
-------> Policy 0 IMPROVED ---------------------
[0.         0.         0.03494066 0.01474878 0.         0.
 0.08308461 0.         0.         0.14678545 0.2427983  0.
 0.         0.24802032 0.5800101  0.        ]
-------- Policy 1 ------------------------------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
-------> Policy 1 IMPROVED ---------------------
[0.         0.         0.02279889 0.00950109 0.         0.
 0.05453427 0.         0.         0.         0.15918618 0.
 0.         0.         0.47615923 0.        ]
-------- Policy 2 ------------------------------
[0.00483051 0.00936909 0.0265529  0.         0.00239941 0.
 0.0793711  0.         0.0009961  0.17325761 0.23839423 0.
 0.         0.3402466  0.62182426 0.        ]
-------> Policy 2 IMPROVED ---------------------


### Step C: Implement Policy Iteration

In [15]:
def policy_iteration(P, nS, nA, gamma=0.9, tol=1e-3):
    """ 
    Run policy iteration for dynamics of P.

    You should use your methods: policy_evaluation() and policy_improvement() here

    Parameters: 
    P, nS, nA, gamma: defined at beginning of file
    tolerance:        tolerance parameter used in policy_evaluation()
    
    Returns: 
    value_function: np.ndarray[nS]
    policy:         np.ndarray[nS]
    """

    value_function = np.zeros(nS)
    policy = np.zeros(nS, dtype=int)

    ############################
    # YOUR IMPLEMENTATION HERE #
    
    while(True):
        
        value_function = policy_evaluation(P, nS, nA, policy, gamma, tol)
        new_policy = policy_improvement(P, nS, nA, value_function, policy, gamma)
        
        if np.array_equal(policy,new_policy): #or np.array_equal(new_value_function,value_function):
            break
            
        policy = new_policy
        
    ############################
    return value_function, policy

#### Call your function for gamma=0.9 and gamma=0.6

In [32]:
V_pi_s, p_pi_s = policy_iteration(env.P, env.nS, env.nA, gamma=0.9, tol=1e-3)
V_pi_s, p_pi_s

(array([0.06476289, 0.05841569, 0.07252933, 0.05378964, 0.08867795,
        0.        , 0.11137269, 0.        , 0.14325408, 0.24628655,
        0.29886952, 0.        , 0.        , 0.37915415, 0.63865133,
        0.        ]),
 array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0]))

In [31]:
V_pi_f, p_pi_f = policy_iteration(env.P, env.nS, env.nA, gamma=0.6, tol=1e-3)
V_pi_f, p_pi_f

(array([0.00069547, 0.00139264, 0.00562142, 0.00171616, 0.00280064,
        0.        , 0.02169924, 0.        , 0.01214942, 0.04755381,
        0.1032758 , 0.        , 0.        , 0.12348492, 0.44745551,
        0.        ]),
 array([1, 3, 2, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0]))

#### What do you observe in terms of impact of gamma on the actions taken by the policy?

*YOUR ANSWER*